In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
# import rasterio
from shapely.geometry import Point
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Clean column names
def clean_columns(df):
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)
    return df

# Load and preprocess 5G DL data
df_dl = pd.read_excel("../data/raw/5G_DL.xlsx", sheet_name="Series Formatted Data")
df_dl = clean_columns(df_dl).dropna(how="all")
df_dl = df_dl[['Longitude', 'Latitude', 'NR_UE_PCI_0', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0']].dropna()

# Load and preprocess 5G UL data
df_ul = pd.read_excel("../data/raw/5G_UL.xlsx", sheet_name="Series Formatted Data")
df_ul = clean_columns(df_ul).dropna(how="all")
df_ul = df_ul[['Longitude', 'Latitude', 'NR_UE_Power_Tx_PUSCH_0']].dropna()

# Merge DL + UL by rounded coordinates
df_dl['lat_round'] = df_dl['Latitude'].round(5)
df_dl['lon_round'] = df_dl['Longitude'].round(5)
df_ul['lat_round'] = df_ul['Latitude'].round(5)
df_ul['lon_round'] = df_ul['Longitude'].round(5)
df_combined = pd.merge(df_dl, df_ul, on=['lat_round', 'lon_round'], suffixes=('_DL', '_UL'))

# Load cell tower info
df_cells = pd.read_excel("../data/raw/İTÜ 5G Hücre Bilgileri.xlsx")
df_cells = clean_columns(df_cells)
df_cells = df_cells[['PCI', 'Longitude', 'Latitude']].dropna().drop_duplicates()
df_cells.rename(columns={'PCI': 'NR_UE_PCI_0'}, inplace=True)

# Merge tower info
df_combined = pd.merge(df_combined, df_cells, on='NR_UE_PCI_0', suffixes=('', '_Tower'))

# Calculate distance to serving tower
df_combined['distance_to_tower'] = df_combined.apply(
    lambda row: geodesic(
        (row['Latitude_DL'], row['Longitude_DL']),
        (row['Latitude_Tower'], row['Longitude_Tower'])
    ).meters, axis=1
)

# Add elevation using raster (change path to your elevation file)
with rasterio.open("../data/raw/maps/ITU_YUKSEKLIK_UTM35NWGS84.asc") as src:
    def get_elevation(lat, lon):
        try:
            x, y = src.index(lon, lat)
            return src.read(1)[x, y]
        except:
            return np.nan

    df_combined['elevation'] = df_combined.apply(
        lambda row: get_elevation(row['Latitude_DL'], row['Longitude_DL']), axis=1)

# Drop rows with missing elevation
df_combined.dropna(subset=['elevation'], inplace=True)

# ============================
# 🎯 MODELING
# ============================

features = [
    'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0',
    'NR_UE_Power_Tx_PUSCH_0', 'distance_to_tower', 'elevation'
]
target = ['Longitude_DL', 'Latitude_DL']

X = df_combined[features].dropna()
y = df_combined.loc[X.index, target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("📉 RMSE (approx meters):", rmse)
print("📈 R² score:", r2)


KeyError: 'Latitude_Tower'

In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from shapely.geometry import Point
import os

# === FILE PATHS ===
dl_file = "../data/raw/5G_DL.xlsx"
ul_file = "../data/raw/5G_UL.xlsx"
scanner_file = "../data/raw/5G_Scanner.xlsx"
cell_info_file = "../data/raw/İTÜ 5G Hücre Bilgileri.xlsx"
elevation_file = "../data/raw/maps/ITU_YUKSEKLIK_UTM35NWGS84.asc"
map_folder = "../data/raw/maps"

# === CLEAN COLUMNS ===
def clean_columns(df):
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)
    return df

# === LOAD + CLEAN DL DATA ===


import pandas as pd

# Load Excel
file_path = "../data/raw/5G_DL.xlsx"
df = pd.read_excel(file_path, sheet_name="Series Formatted Data")

# Step 1: Drop fully empty rows/columns
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

# Step 2: Clean column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)

# Step 3: Define important columns (serving + neighbor cells)
important_cols = [
    'Longitude', 'Latitude',
    'NR_UE_PCI_0', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0',
    'NR_UE_Nbr_PCI_0', 'NR_UE_Nbr_PCI_1', 'NR_UE_Nbr_PCI_2', 'NR_UE_Nbr_PCI_3', 'NR_UE_Nbr_PCI_4',
    'NR_UE_Nbr_RSRP_0', 'NR_UE_Nbr_RSRP_1', 'NR_UE_Nbr_RSRP_2', 'NR_UE_Nbr_RSRP_3', 'NR_UE_Nbr_RSRP_4',
    'NR_UE_Nbr_RSRQ_0', 'NR_UE_Nbr_RSRQ_1', 'NR_UE_Nbr_RSRQ_2', 'NR_UE_Nbr_RSRQ_3', 'NR_UE_Nbr_RSRQ_4',
    'NR_UE_Timing_Advance', 'NR_UE_Pathloss_DL_0', 'NR_UE_Power_Tx_PUSCH_0'
]
df = df[[col for col in important_cols if col in df.columns]]

# Step 4: Drop rows with no location
df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

# Step 5: Drop rows where all main serving signal values are missing
serving_cols = ['NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0']
df.dropna(subset=serving_cols, how='all', inplace=True)

# Step 6: Replace remaining NaNs with a sentinel value (-999)
df.fillna(-999, inplace=True)

# Step 7: Reset index
df.reset_index(drop=True, inplace=True)

# Step 8: Summary
print("✅ Final cleaned shape:", df.shape)
print("\n📊 Remaining columns:\n", df.columns.tolist())
print("\n📈 Statistical summary:\n", df.describe())







# === LOAD + CLEAN UL DATA ===
import pandas as pd

# Step 1: Load the Series Formatted Data sheet
file_path = "../data/raw/5G_UL.xlsx"
df = pd.read_excel(file_path, sheet_name="Series Formatted Data")

# Step 2: Drop completely empty rows and columns
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

# Step 3: Clean up column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)

# Step 4: Check shape and column names
print("Initial shape:", df.shape)
print("Columns:", df.columns.tolist())

# Step 5: Drop rows where Latitude/Longitude are missing (if present)
if 'Longitude' in df.columns and 'Latitude' in df.columns:
    df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

# Step 6: Drop columns with 100% missing values
df = df.dropna(axis=1, how='all')

# Step 7: Drop columns with very low data (e.g., < 1% non-null)
threshold = 0.01 * len(df)
df = df.dropna(axis=1, thresh=threshold)




# === LOAD + CLEAN SCANNER DATA ===

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Load the Series Formatted Data
file_path = "../data/raw/5G_Scanner.xlsx"
df = pd.read_excel(file_path, sheet_name="Series Formatted Data")

# Step 2: Drop fully empty rows/columns
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

# Step 3: Clean column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)

# Step 4: Remove rows missing Latitude/Longitude if present
if 'Longitude' in df.columns and 'Latitude' in df.columns:
    df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

# Step 5: Drop columns with 100% missing data
df = df.dropna(axis=1, how='all')

# Step 6: Drop low-information columns (<1% populated)
threshold = 0.01 * len(df)
df = df.dropna(axis=1, thresh=threshold)

# Step 7: Summary
print("✅ Cleaned shape:", df.shape)
print("\n📊 Remaining columns:", df.columns.tolist())
print("\n📈 Description:\n", df.describe())






# === LOAD + CLEAN CELL INFO ===
df_cells = pd.read_excel(cell_info_file)
df_cells = clean_columns(df_cells)
df_cells = df_cells[['PCI', 'Longitude', 'Latitude']].dropna().drop_duplicates()

# === ADD ELEVATION FUNCTION ===
def add_elevation(df, lat_col='Latitude', lon_col='Longitude'):
    with rasterio.open(elevation_file) as src:
        def get_elev(lat, lon):
            try:
                row, col = src.index(lon, lat)
                return src.read(1)[row, col]
            except:
                return -9999
        df['elevation'] = df.apply(lambda row: get_elev(row[lat_col], row[lon_col]), axis=1)
    return df

df_dl = add_elevation(df_dl)
df_ul = add_elevation(df_ul)
df_scanner = add_elevation(df_scanner)

# === OPTIONAL: LOAD VECTOR MAP LAYERS (GeoDataFrames) ===
def load_layer(name):
    file_path = os.path.join(map_folder, name)
    return gpd.read_file(file_path) if os.path.exists(file_path) else None

gdf_buildings = load_layer("ITU_3DBINA_EPSG4326.shp")
gdf_roads = load_layer("ITU_ULASIMAGI_EPSG4326.shp")
gdf_water = load_layer("ITU_SUKUTLESI_EPSG4326.shp")
gdf_vegetation = load_layer("ITU_3DVEGETATION_EPSG4326.shp")
gdf_walls = load_layer("ITU_SINIRDUVAR_EPSG4326.shp")

# === DONE ===
print("✅ DL Data:", df_dl.shape)
print("✅ UL Data:", df_ul.shape)
print("✅ Scanner Data:", df_scanner.shape)


✅ Final cleaned shape: (1942, 24)

📊 Remaining columns:
 ['Longitude', 'Latitude', 'NR_UE_PCI_0', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0', 'NR_UE_Nbr_PCI_0', 'NR_UE_Nbr_PCI_1', 'NR_UE_Nbr_PCI_2', 'NR_UE_Nbr_PCI_3', 'NR_UE_Nbr_PCI_4', 'NR_UE_Nbr_RSRP_0', 'NR_UE_Nbr_RSRP_1', 'NR_UE_Nbr_RSRP_2', 'NR_UE_Nbr_RSRP_3', 'NR_UE_Nbr_RSRP_4', 'NR_UE_Nbr_RSRQ_0', 'NR_UE_Nbr_RSRQ_1', 'NR_UE_Nbr_RSRQ_2', 'NR_UE_Nbr_RSRQ_3', 'NR_UE_Nbr_RSRQ_4', 'NR_UE_Timing_Advance', 'NR_UE_Pathloss_DL_0', 'NR_UE_Power_Tx_PUSCH_0']

📈 Statistical summary:
          Longitude     Latitude  NR_UE_PCI_0  NR_UE_RSRP_0  NR_UE_RSRQ_0  \
count  1942.000000  1942.000000  1942.000000   1942.000000   1942.000000   
mean     29.023036    41.105459    47.819258    -92.609629    -12.855458   
std       0.003757     0.002563    22.482686     20.777432      3.085778   
min      29.015340    41.098950     3.000000   -150.600000    -37.600000   
25%      29.020555    41.104602    30.000000   -115.800000    -14.275000   
50%  

In [ ]:
!pip install rasterio

^C


   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.5 MB ? eta -:--:--
    --------------------------------------- 0.5/25.5 MB 621.2 kB/s eta 0:00:41
    --------------------------------------- 0.5/25.5 MB 621.2 kB/s eta 0:00:41
    --------------------------------------- 0.5/25.5 MB 621.2 kB/s eta 0:00:41
   - -------------------------------------- 0.8/25.5 MB 500.8 kB/s eta 0:00:50
   - -------------------------------------- 0.8/25.5 MB 500.8 kB/s eta 0:00:50
   - -------------------------------------- 0.8/25.5 MB 500.8 kB/s eta 0:00:50
   - -------------------------------------- 0.8/25.5 MB 500.8 kB/s eta 0:00:50
   - ----------------------

In [27]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from shapely.geometry import Point
import os

# === FILE PATHS ===
dl_file = "../data/raw/5G_DL.xlsx"
ul_file = "../data/raw/5G_UL.xlsx"
scanner_file = "../data/raw/5G_Scanner.xlsx"
cell_info_file = "../data/raw/İTÜ 5G Hücre Bilgileri.xlsx"
elevation_file = "../data/raw/maps/ITU_YUKSEKLIK_UTM35NWGS84.asc"
map_folder = "../data/raw/maps"


In [28]:
def clean_columns(df):
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)
    return df

def add_elevation(df, lat_col='Latitude', lon_col='Longitude'):
    with rasterio.open(elevation_file) as src:
        def get_elev(lat, lon):
            try:
                row, col = src.index(lon, lat)
                return src.read(1)[row, col]
            except:
                return -9999
        df['elevation'] = df.apply(lambda row: get_elev(row[lat_col], row[lon_col]), axis=1)
    return df

def load_layer(name):
    file_path = os.path.join(map_folder, name)
    return gpd.read_file(file_path) if os.path.exists(file_path) else None


In [29]:
df_dl = pd.read_excel(dl_file, sheet_name="Series Formatted Data")
df_dl = clean_columns(df_dl).dropna(how="all")

important_cols = [
    'Longitude', 'Latitude', 'NR_UE_PCI_0', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0',
    'NR_UE_Nbr_PCI_0', 'NR_UE_Nbr_PCI_1', 'NR_UE_Nbr_PCI_2', 'NR_UE_Nbr_PCI_3', 'NR_UE_Nbr_PCI_4',
    'NR_UE_Nbr_RSRP_0', 'NR_UE_Nbr_RSRP_1', 'NR_UE_Nbr_RSRP_2', 'NR_UE_Nbr_RSRP_3', 'NR_UE_Nbr_RSRP_4',
    'NR_UE_Nbr_RSRQ_0', 'NR_UE_Nbr_RSRQ_1', 'NR_UE_Nbr_RSRQ_2', 'NR_UE_Nbr_RSRQ_3', 'NR_UE_Nbr_RSRQ_4',
    'NR_UE_Timing_Advance', 'NR_UE_Pathloss_DL_0', 'NR_UE_Power_Tx_PUSCH_0'
]
df_dl = df_dl[[col for col in important_cols if col in df_dl.columns]]
df_dl.dropna(subset=['Longitude', 'Latitude'], inplace=True)
df_dl.dropna(subset=['NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0'], how='all', inplace=True)
df_dl.fillna(-999, inplace=True)
df_dl = add_elevation(df_dl)


In [11]:
df_ul = pd.read_excel(ul_file, sheet_name="Series Formatted Data")
df_ul = clean_columns(df_ul).dropna(how="all")
df_ul.dropna(subset=['Longitude', 'Latitude'], inplace=True)
df_ul = df_ul.dropna(axis=1, how='all')

# Remove columns with <1% data
ul_thresh = 0.01 * len(df_ul)
df_ul = df_ul.dropna(axis=1, thresh=ul_thresh)
df_ul.fillna(-999, inplace=True)
df_ul = add_elevation(df_ul)


In [30]:
import pandas as pd

# Step 1: Load the Series Formatted Data sheet
file_path = "../data/raw/5G_UL.xlsx"
df_ul = pd.read_excel(file_path, sheet_name="Series Formatted Data")

# Step 2: Drop completely empty rows and columns
df_ul.dropna(axis=0, how='all', inplace=True)
df_ul.dropna(axis=1, how='all', inplace=True)

# Step 3: Clean up column names
df_ul.columns = df_ul.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)

# Step 4: Check shape and column names
print("Initial shape:", df_ul.shape)
print("Columns:", df_ul.columns.tolist())

# Step 5: Drop rows where Latitude/Longitude are missing (if present)
if 'Longitude' in df_ul.columns and 'Latitude' in df_ul.columns:
    df_ul.dropna(subset=['Longitude', 'Latitude'], inplace=True)

# Step 6: Drop columns with 100% missing values
df_ul = df_ul.dropna(axis=1, how='all')

# Step 7: Drop columns with very low data (e.g., < 1% non-null)
threshold = 0.01 * len(df_ul)
df_ul = df_ul.dropna(axis=1, thresh=threshold)

# Step 8: Basic summary
print("\n✅ Cleaned shape:", df_ul.shape)
print("\n📊 Remaining columns:", df_ul.columns.tolist())
print("\n📈 Statistical summary:\n", df_ul.describe())


Initial shape: (59808, 55)
Columns: ['Message', 'Time', 'Longitude', 'Latitude', 'Technology_Mode', 'NR_UE_PCI_0', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0', 'NR_UE_Nbr_PCI_0', 'NR_UE_Nbr_PCI_1', 'NR_UE_Nbr_PCI_2', 'NR_UE_Nbr_PCI_3', 'NR_UE_Nbr_PCI_4', 'NR_UE_Nbr_RSRP_0', 'NR_UE_Nbr_RSRP_1', 'NR_UE_Nbr_RSRP_2', 'NR_UE_Nbr_RSRP_3', 'NR_UE_Nbr_RSRP_4', 'NR_UE_Nbr_RSRQ_0', 'NR_UE_Nbr_RSRQ_1', 'NR_UE_Nbr_RSRQ_2', 'NR_UE_Nbr_RSRQ_3', 'NR_UE_Nbr_RSRQ_4', 'NR_UE_Timing_Advance', 'NR_UE_Pathloss_DL_0', 'NR_UE_Throughput_PDCP_DL', 'NR_UE_NACK_Rate_DL_0', 'NR_UE_Ack_As_Nack_DL_0', 'NR_UE_MCS_DL_0', 'NR_UE_RB_Num_DL_0', 'NR_UE_Modulation_Avg_DL_0', 'NR_UE_RI_DL_0', 'NR_UE_BLER_DL_0', 'NR_UE_CCE_AggregationLev_0', 'NR_UE_Power_Tx_PUSCH_0', 'NR_UE_Power_Tx_PRACH_0', 'NR_UE_NACK_Rate_UL_0', 'NR_UE_RACH_Attempt', 'NR_UE_RACH_OK', 'NR_UE_RACH_Fail', 'NR_UE_RACH_Procedure_Count', 'NR_UE_RRCReEstAttempt', 'NR_UE_RRCReEstFail', 'NR_UE_RRCReEst_EndResult', 'NR_UE_RRCConnectionAttempt', 'NR_UE_RRCConn

In [5]:
df_scanner = pd.read_excel(scanner_file, sheet_name="Series Formatted Data")
df_scanner = clean_columns(df_scanner).dropna(how="all")
df_scanner.dropna(subset=['Longitude', 'Latitude'], inplace=True)
df_scanner = df_scanner.dropna(axis=1, how='all')

scanner_thresh = 0.01 * len(df_scanner)
df_scanner = df_scanner.dropna(axis=1, thresh=scanner_thresh)
df_scanner.fillna(-999, inplace=True)
df_scanner = add_elevation(df_scanner)


In [31]:
df_cells = pd.read_excel(cell_info_file)
df_cells = clean_columns(df_cells)
df_cells = df_cells[['PCI', 'Longitude', 'Latitude']].dropna().drop_duplicates()


In [7]:
gdf_buildings = load_layer("ITU_3DBINA_EPSG4326.shp")
gdf_roads = load_layer("ITU_ULASIMAGI_EPSG4326.shp")
gdf_water = load_layer("ITU_SUKUTLESI_EPSG4326.shp")
gdf_vegetation = load_layer("ITU_3DVEGETATION_EPSG4326.shp")
gdf_walls = load_layer("ITU_SINIRDUVAR_EPSG4326.shp")


In [32]:
print("✅ DL Data:", df_dl.shape)
print("✅ UL Data:", df_ul.shape)
print("✅ Scanner Data:", df_scanner.shape)
print("✅ Cell Tower Data:", df_cells.shape)


✅ DL Data: (1942, 25)
✅ UL Data: (59806, 37)
✅ Scanner Data: (22390, 29)
✅ Cell Tower Data: (9, 3)


In [33]:
df_dl.head()

,Longitude,Latitude,NR_UE_PCI_0,NR_UE_RSRP_0,NR_UE_RSRQ_0,NR_UE_SINR_0,NR_UE_Nbr_PCI_0,NR_UE_Nbr_PCI_1,NR_UE_Nbr_PCI_2,NR_UE_Nbr_PCI_3,...,NR_UE_Nbr_RSRP_4,NR_UE_Nbr_RSRQ_0,NR_UE_Nbr_RSRQ_1,NR_UE_Nbr_RSRQ_2,NR_UE_Nbr_RSRQ_3,NR_UE_Nbr_RSRQ_4,NR_UE_Timing_Advance,NR_UE_Pathloss_DL_0,NR_UE_Power_Tx_PUSCH_0,elevation
27,29.02949,41.10723,48.0,-84.4,-11.0,6.9,76.0,-999.0,-999.0,-999.0,...,-999.0,-16.1,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-9999
82,29.02949,41.10723,48.0,-85.1,-11.2,-999.0,76.0,-999.0,-999.0,-999.0,...,-999.0,-16.2,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-9999
111,29.02949,41.10723,48.0,-86.2,-11.5,5.6,76.0,-999.0,-999.0,-999.0,...,-999.0,-17.5,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-9999
142,29.02949,41.10723,48.0,-86.7,-11.6,9.9,76.0,-999.0,-999.0,-999.0,...,-999.0,-16.3,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-9999
173,29.02949,41.10723,48.0,-85.8,-10.6,13.4,76.0,-999.0,-999.0,-999.0,...,-999.0,-17.8,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-9999


In [19]:
df_cells.head(10)

,NR_UE_PCI_0,Longitude,Latitude
0,30,29.023367,41.107341
1,40,29.022665,41.107087
2,59,29.028122,41.108086
3,48,29.028122,41.108086
4,68,29.027833,41.105469
5,76,29.027833,41.105469
6,3,29.021747,41.104342
7,13,29.021747,41.104342
8,23,29.020503,41.105842


In [34]:
# Helper: Convert PCI columns to Lat/Lon using df_cells
def replace_pci_with_coords(df, pci_cols, suffix=""):
    for col in pci_cols:
        if col not in df.columns:
            continue
        new_lat_col = col.replace("PCI", "LAT")
        new_lon_col = col.replace("PCI", "LON")
        
        df[new_lat_col] = df[col].map(df_cells.set_index('PCI')['Latitude'])
        df[new_lon_col] = df[col].map(df_cells.set_index('PCI')['Longitude'])
    return df



# Step 2: Get all PCI columns from DL and UL datasets
dl_pci_cols = [col for col in df_dl.columns if "PCI" in col]
ul_pci_cols = [col for col in df_ul.columns if "PCI" in col]

# Step 3: Replace PCI values with LAT/LON in both DL and UL
df_dl = replace_pci_with_coords(df_dl, dl_pci_cols)
df_ul = replace_pci_with_coords(df_ul, ul_pci_cols)

# Step 4: Drop the original PCI columns if you don't want them
df_dl.drop(columns=dl_pci_cols, inplace=True)
df_ul.drop(columns=ul_pci_cols, inplace=True)

# ✅ Done
print("DL columns now:", df_dl.columns.tolist())
print("UL columns now:", df_ul.columns.tolist())

DL columns now: ['Longitude', 'Latitude', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0', 'NR_UE_Nbr_RSRP_0', 'NR_UE_Nbr_RSRP_1', 'NR_UE_Nbr_RSRP_2', 'NR_UE_Nbr_RSRP_3', 'NR_UE_Nbr_RSRP_4', 'NR_UE_Nbr_RSRQ_0', 'NR_UE_Nbr_RSRQ_1', 'NR_UE_Nbr_RSRQ_2', 'NR_UE_Nbr_RSRQ_3', 'NR_UE_Nbr_RSRQ_4', 'NR_UE_Timing_Advance', 'NR_UE_Pathloss_DL_0', 'NR_UE_Power_Tx_PUSCH_0', 'elevation', 'NR_UE_LAT_0', 'NR_UE_LON_0', 'NR_UE_Nbr_LAT_0', 'NR_UE_Nbr_LON_0', 'NR_UE_Nbr_LAT_1', 'NR_UE_Nbr_LON_1', 'NR_UE_Nbr_LAT_2', 'NR_UE_Nbr_LON_2', 'NR_UE_Nbr_LAT_3', 'NR_UE_Nbr_LON_3', 'NR_UE_Nbr_LAT_4', 'NR_UE_Nbr_LON_4']
UL columns now: ['Message', 'Time', 'Longitude', 'Latitude', 'Technology_Mode', 'NR_UE_RSRP_0', 'NR_UE_RSRQ_0', 'NR_UE_SINR_0', 'NR_UE_Nbr_RSRP_0', 'NR_UE_Nbr_RSRQ_0', 'NR_UE_Pathloss_DL_0', 'NR_UE_Throughput_PDCP_DL', 'NR_UE_NACK_Rate_DL_0', 'NR_UE_Ack_As_Nack_DL_0', 'NR_UE_MCS_DL_0', 'NR_UE_RB_Num_DL_0', 'NR_UE_Modulation_Avg_DL_0', 'NR_UE_RI_DL_0', 'NR_UE_BLER_DL_0', 'NR_UE_CCE_AggregationLev_0', '

In [35]:
df_dl.head()

,Longitude,Latitude,NR_UE_RSRP_0,NR_UE_RSRQ_0,NR_UE_SINR_0,NR_UE_Nbr_RSRP_0,NR_UE_Nbr_RSRP_1,NR_UE_Nbr_RSRP_2,NR_UE_Nbr_RSRP_3,NR_UE_Nbr_RSRP_4,...,NR_UE_Nbr_LAT_0,NR_UE_Nbr_LON_0,NR_UE_Nbr_LAT_1,NR_UE_Nbr_LON_1,NR_UE_Nbr_LAT_2,NR_UE_Nbr_LON_2,NR_UE_Nbr_LAT_3,NR_UE_Nbr_LON_3,NR_UE_Nbr_LAT_4,NR_UE_Nbr_LON_4
27,29.02949,41.10723,-84.4,-11.0,6.9,-96.9,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,29.02949,41.10723,-85.1,-11.2,-999.0,-95.3,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,29.02949,41.10723,-86.2,-11.5,5.6,-96.6,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,29.02949,41.10723,-86.7,-11.6,9.9,-96.1,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,29.02949,41.10723,-85.8,-10.6,13.4,-100.4,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# ✅ Step 2: Apply to your DL and UL DataFrames

# Rename PCI column in cell info if needed
df_cells = df_cells.rename(columns={'PCI': 'PCI'})  # keep as 'PCI'

# DL PCI columns
dl_pci_cols = [col for col in df_dl.columns if "PCI" in col]
df_dl = pci_to_coords(df_dl, dl_pci_cols, df_cells)

# UL PCI columns
ul_pci_cols = [col for col in df_ul.columns if "PCI" in col]
df_ul = pci_to_coords(df_ul, ul_pci_cols, df_cells)

# ✅ Done
print("✅ DL shape:", df_dl.shape)
print("✅ UL shape:", df_ul.shape)
print("✅ Sample DL columns:", df_dl.columns.tolist()[:10])
print("✅ Sample UL columns:", df_ul.columns.tolist()[:10])


KeyError: "None of ['PCI'] are in the columns"

In [16]:
df_dl.head()

,Longitude,Latitude,NR_UE_RSRP_0,NR_UE_RSRQ_0,NR_UE_SINR_0,NR_UE_Nbr_RSRP_0,NR_UE_Nbr_RSRP_1,NR_UE_Nbr_RSRP_2,NR_UE_Nbr_RSRP_3,NR_UE_Nbr_RSRP_4,...,NR_UE_Nbr_LAT_0,NR_UE_Nbr_LON_0,NR_UE_Nbr_LAT_1,NR_UE_Nbr_LON_1,NR_UE_Nbr_LAT_2,NR_UE_Nbr_LON_2,NR_UE_Nbr_LAT_3,NR_UE_Nbr_LON_3,NR_UE_Nbr_LAT_4,NR_UE_Nbr_LON_4
27,29.02949,41.10723,-84.4,-11.0,6.9,-96.9,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,29.02949,41.10723,-85.1,-11.2,-999.0,-95.3,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,29.02949,41.10723,-86.2,-11.5,5.6,-96.6,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,29.02949,41.10723,-86.7,-11.6,9.9,-96.1,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,29.02949,41.10723,-85.8,-10.6,13.4,-100.4,-999.0,-999.0,-999.0,-999.0,...,41.105469,29.027833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
